In [ ]:
import getpass
import pandas as pd
import numpy as np
from termcolor import colored
import import_ipynb
import _client
import _stream
import _historical

In [ ]:
class Trader():
    '''Binance Client Real Time Trader'''
    
    def __init__(self):
        testnet = input('1/0 Testnet ')
        api_key = getpass.getpass('API KEY ')
        secret_key = getpass.getpass('API SECRET ')
        
        assert (api_key and secret_key and testnet), "everything is required"
        
        self.api_key = api_key
        self.secret_key = secret_key
        self.testnet = bool(int(testnet))
        
        self.__connect_client()
        
        self.valid_intervals = ['1m','3m','5m','15m','30m','1h','2h','4h','6h','8h','12h','1d','3d','1w','1M']

    def get_historical_data(self):
        symbol, interval = self.__ask_symbol_interval_inputs()
        
        days = int(input('DAYS '))
        assert days > 0, 'DAYS is not valid'
        
        return _historical.get_most_recent(self.client, symbol, interval, days)
    
    def start_live_trading(self):
        symbol, interval = self.__ask_symbol_interval_inputs()
        print('Starting Live Trading...')
        self.live_trading_options = {'symbol': symbol, 'interval': interval}
        self.twm, self.live_data = _stream.start_kline_socket(symbol=symbol, interval=interval, cb=self.__stream_candles)
        
    def stop(self):
        print('Stopping Live Trading Session...')
        self.twm.stop()
    
    # PRIVATE
    def __connect_client(self):
        client = _client.get_client(self.api_key, self.secret_key, self.testnet)
        assert client, 'Authentication failed!'
        self.client = client
    
    def __ask_symbol_interval_inputs(self):
        symbol = input('SYMBOL (default:BTCUSDT) ')
        symbol = symbol or 'BTCUSDT'
        
        print("\nValid intervals: {}".format(self.valid_intervals))
        interval = input('INTERVAL (default:1m) ')
        interval = interval or '1m'
        
        assert interval in self.valid_intervals, 'INTERVAL is not valid'

        return symbol, interval
              
    def __stream_candles(self, msg):
        event_time = pd.to_datetime(msg['E'], unit='ms')
        start_time = pd.to_datetime(msg['k']['t'], unit='ms')

        first    = float(msg['k']['o'])
        high     = float(msg['k']['h'])
        low      = float(msg['k']['l'])
        close    = float(msg['k']['c'])
        volume   = float(msg['k']['v'])
        complete =  bool(msg['k']['x'])

        print("Indicator: {} | Time: {} | Price: {}".format('.' if complete == False else '|', event_time, close))

        self.live_data.loc[start_time] = [first, high, low, close, volume, complete]
        
        mean_vol = self.live_data['Volume'].mean()
        mean_high = self.live_data['High'].mean()
        
        if(volume > mean_vol*1.5 and close > mean_high*1.5):
            trading_options = self.live_trading_options
            print('GOING LONG WITH QUANTITY 0.001...')
            self.client.create_order(symbol=trading_options['symbol'], side='BUY', type='MARKET', quoteOrderQty=0.001)        

In [ ]:
trader = Trader()

In [ ]:
trader.get_historical_data()

In [ ]:
trader.start_live_trading()

In [ ]:
trader.stop()